# Association rules
Apriori algorithm

In [1]:
%load_ext sql

/Users/fmcquillan/anaconda/lib/python2.7/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated since IPython 4.0. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/Users/fmcquillan/anaconda/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [2]:
# Greenplum Database 5.x on GCP (PM demo machine) - direct external IP access
#%sql postgresql://gpadmin@34.67.65.96:5432/madlib

# Greenplum Database 5.x on GCP - via tunnel
%sql postgresql://gpadmin@localhost:8000/madlib
        
# PostgreSQL local
#%sql postgresql://fmcquillan@localhost:5432/madlib

u'Connected: gpadmin@madlib'

In [3]:
%sql select madlib.version();
#%sql select version();

1 rows affected.


version
"MADlib version: 1.17-dev, git revision: rel/v1.16-54-gec5614f, cmake configuration time: Wed Dec 18 17:08:05 UTC 2019, build type: release, build system: Linux-3.10.0-1062.4.3.el7.x86_64, C compiler: gcc 4.8.5, C++ compiler: g++ 4.8.5"


# 1.  Load data

In [4]:
%%sql 
DROP TABLE IF EXISTS test_data;
CREATE TABLE test_data (
    trans_id INT,
    product TEXT
);

INSERT INTO test_data VALUES (1, 'beer');
INSERT INTO test_data VALUES (1, 'diapers');
INSERT INTO test_data VALUES (1, 'chips');
INSERT INTO test_data VALUES (2, 'beer');
INSERT INTO test_data VALUES (2, 'diapers');
INSERT INTO test_data VALUES (3, 'beer');
INSERT INTO test_data VALUES (3, 'diapers');
INSERT INTO test_data VALUES (4, 'beer');
INSERT INTO test_data VALUES (4, 'chips');
INSERT INTO test_data VALUES (5, 'beer');
INSERT INTO test_data VALUES (6, 'beer');
INSERT INTO test_data VALUES (6, 'diapers');
INSERT INTO test_data VALUES (6, 'chips');
INSERT INTO test_data VALUES (7, 'beer');
INSERT INTO test_data VALUES (7, 'diapers');

SELECT * FROM test_data ORDER BY trans_id;

Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
15 rows affected.


trans_id,product
1,beer
1,chips
1,diapers
2,beer
2,diapers
3,beer
3,diapers
4,beer
4,chips
5,beer


# 2.  Generate rules
Let min(support)=.25 and min(confidence)=.5, and the output schema is set to NULL indicating output to the current schema. In this example we set verbose to TRUE so that we have some insight into progress of the function. We can now generate association rules as follows:

In [5]:
%%sql
SELECT * FROM madlib.assoc_rules( .25,            -- Support
                                  .5,             -- Confidence
                                  'trans_id',     -- Transaction id col
                                  'product',      -- Product col
                                  'test_data',    -- Input data
                                  NULL,           -- Output schema
                                  TRUE            -- Verbose output
                                );

1 rows affected.


output_schema,output_table,total_rules,total_time
public,assoc_rules,7,0:00:00.926710


The association rules are stored in the assoc_rules table:

In [6]:
%%sql
SELECT * FROM assoc_rules
ORDER BY support DESC, confidence DESC;

7 rows affected.


ruleid,pre,post,count,support,confidence,lift,conviction
1,[u'diapers'],[u'beer'],5,0.714285714286,1.0,1.0,0.0
6,[u'beer'],[u'diapers'],5,0.714285714286,0.714285714286,1.0,1.0
4,[u'chips'],[u'beer'],3,0.428571428571,1.0,1.0,0.0
2,"[u'diapers', u'chips']",[u'beer'],2,0.285714285714,1.0,1.0,0.0
7,[u'chips'],[u'diapers'],2,0.285714285714,0.666666666667,0.933333333333,0.857142857143
3,"[u'beer', u'chips']",[u'diapers'],2,0.285714285714,0.666666666667,0.933333333333,0.857142857143
5,[u'chips'],"[u'diapers', u'beer']",2,0.285714285714,0.666666666667,0.933333333333,0.857142857143


# 3. Limit max itemset size
Limit association rules generated from itemsets of size at most 2. This parameter is a good way to reduce long run times.

In [7]:
%%sql 
SELECT * FROM madlib.assoc_rules( .25,            -- Support
                                  .5,             -- Confidence
                                  'trans_id',     -- Transaction id col
                                  'product',      -- Product col
                                  'test_data',    -- Input data
                                  NULL,           -- Output schema
                                  TRUE,           -- Verbose output
                                  2               -- Max itemset size
                                );

1 rows affected.


output_schema,output_table,total_rules,total_time
public,assoc_rules,4,0:00:00.746039


The association rules are again stored in the assoc_rules table:

In [8]:
%%sql
SELECT * FROM assoc_rules
ORDER BY support DESC, confidence DESC;

4 rows affected.


ruleid,pre,post,count,support,confidence,lift,conviction
2,[u'diapers'],[u'beer'],5,0.714285714286,1.0,1.0,0.0
3,[u'beer'],[u'diapers'],5,0.714285714286,0.714285714286,1.0,1.0
1,[u'chips'],[u'beer'],3,0.428571428571,1.0,1.0,0.0
4,[u'chips'],[u'diapers'],2,0.285714285714,0.666666666667,0.933333333333,0.857142857143


# 4. Filter rules
Post-processing can now be done on the output table in the case that you want to filter the results. For example, if you want any single item on the left hand side and a particular item on the right hand side:

In [9]:
%%sql
SELECT * FROM assoc_rules WHERE array_upper(pre,1) = 1 AND post = array['beer'];

2 rows affected.


ruleid,pre,post,count,support,confidence,lift,conviction
1,[u'chips'],[u'beer'],3,0.428571428571,1.0,1.0,0.0
2,[u'diapers'],[u'beer'],5,0.714285714286,1.0,1.0,0.0


# 5. Limit RHS
Limit the size of right hand side to 1. This parameter is a good way to reduce long run times.

In [10]:
%%sql
SELECT * FROM madlib.assoc_rules( .25,            -- Support
                                  .5,             -- Confidence
                                  'trans_id',     -- Transaction id col
                                  'product',      -- Product col
                                  'test_data',    -- Input data
                                  NULL,           -- Output schema
                                  TRUE,           -- Verbose output
                                  NULL,           -- Max itemset size
                                  NULL,           -- Max LHS size
                                  1               -- Max RHS size
                                );

1 rows affected.


output_schema,output_table,total_rules,total_time
public,assoc_rules,6,0:00:00.830826


The association rules are again stored in the assoc_rules table:

In [11]:
%%sql
SELECT * FROM assoc_rules
ORDER BY support DESC, confidence DESC;

6 rows affected.


ruleid,pre,post,count,support,confidence,lift,conviction
1,[u'diapers'],[u'beer'],5,0.714285714286,1.0,1.0,0.0
5,[u'beer'],[u'diapers'],5,0.714285714286,0.714285714286,1.0,1.0
4,[u'chips'],[u'beer'],3,0.428571428571,1.0,1.0,0.0
2,"[u'diapers', u'chips']",[u'beer'],2,0.285714285714,1.0,1.0,0.0
6,[u'chips'],[u'diapers'],2,0.285714285714,0.666666666667,0.933333333333,0.857142857143
3,"[u'chips', u'beer']",[u'diapers'],2,0.285714285714,0.666666666667,0.933333333333,0.857142857143
